: 

In [ ]:
import pandas as pd
import ieee738
import math
from ieee738.ieee738 import ConductorParams, Conductor
import numpy as np
import matplotlib.pyplot as plt
import json

In [3]:
lines = pd.read_csv("hawaii40_osu/csv/lines.csv")
buses = pd.read_csv("hawaii40_osu/csv/buses.csv")
flows = pd.read_csv("hawaii40_osu/line_flows_nominal.csv")
cond_lib = pd.read_csv("ieee738/conductor_library.csv")
cond_lib.head()



,ConductorName,RES_25C,RES_50C,CDRAD_in,CDGMR_ft
0,3/0 ACSR 6/1 PIGEON,0.5405,0.6638,0.2510,0.0161
1,4/0 ACSR 6/1 PENGUIN,0.4288,0.5446,0.2815,0.0180
2,336.4 ACSR 30/7 ORIOLE,0.2708,0.2974,0.3705,0.0255
3,556.5 ACSR 26/7 DOVE,0.1655,0.1816,0.4635,0.0313
4,795 ACSR 26/7 DRAKE,0.1166,0.1278,0.5540,0.0374


In [8]:
df = lines.merge(flows, on="name", how="left")

# Join voltage info from bus0
df = df.merge(
    buses[["name", "v_nom"]].rename(columns={"name": "bus0", "v_nom": "v_nom_kV"}),
    on="bus0",
    how="left"
)

df = df.merge(
    cond_lib[["RES_25C", "RES_50C", "CDRAD_in", "ConductorName"]],
    left_on="conductor",
    right_on="ConductorName",
    how="left"
)

# --- 4. Clean up column names ---
df.rename(columns={
    "s_nom": "s_nom_MVA",
    "p0_nominal": "p0_nominal_MVA",
    "MOT": "MOT_C"
}, inplace=True)

df = df.drop(columns=["ConductorName", "x", "r", "b", "status", "v_ang_min", "v_ang_max", "original_index"])

df["RLo_ohm_per_ft"] = df["RES_25C"] / 5280.0
df["RHi_ohm_per_ft"] = df["RES_50C"] / 5280.0
df["Diameter_in"]    = 2.0 * df["CDRAD_in"] 


# --- 5. Verify ---

print(df.columns)
print(f"Total lines: {len(df)}")

Index(['name', 'bus0', 'bus1', 'ckt', 'branch_name', 'bus0_name', 'bus1_name',
       's_nom_MVA', 'conductor', 'MOT_C', 'p0_nominal_MVA', 'v_nom_kV',
       'RES_25C', 'RES_50C', 'CDRAD_in', 'RLo_ohm_per_ft', 'RHi_ohm_per_ft',
       'Diameter_in'],
      dtype='object')
Total lines: 77


In [30]:
df.head()

,name,bus0,bus1,ckt,branch_name,bus0_name,bus1_name,s_nom_MVA,conductor,MOT_C,p0_nominal_MVA,v_nom_kV,RES_25C,RES_50C,CDRAD_in,RLo_ohm_per_ft,RHi_ohm_per_ft,Diameter_in
0,L0,1,5,1,ALOHA138 (1) TO HONOLULU138 (5) CKT 1,ALOHA138,HONOLULU138,228,795 ACSR 26/7 DRAKE,80,79.168558,138.0,0.1166,0.1278,0.5540,0.000022,0.000024,1.108
1,L1,1,5,2,ALOHA138 (1) TO HONOLULU138 (5) CKT 2,ALOHA138,HONOLULU138,228,795 ACSR 26/7 DRAKE,80,79.168558,138.0,0.1166,0.1278,0.5540,0.000022,0.000024,1.108
2,L2,1,22,1,ALOHA138 (1) TO WAIPAHU138 (22) CKT 1,ALOHA138,WAIPAHU138,172,556.5 ACSR 26/7 DOVE,75,22.449716,138.0,0.1655,0.1816,0.4635,0.000031,0.000034,0.927
3,L3,1,25,1,ALOHA138 (1) TO EWA BEACH138 (25) CKT 1,ALOHA138,EWA BEACH138,215,795 ACSR 26/7 DRAKE,85,66.694086,138.0,0.1166,0.1278,0.5540,0.000022,0.000024,1.108
4,L4,1,25,2,ALOHA138 (1) TO EWA BEACH138 (25) CKT 2,ALOHA138,EWA BEACH138,215,795 ACSR 26/7 DRAKE,85,66.694086,138.0,0.1166,0.1278,0.5540,0.000022,0.000024,1.108


In [ ]:

df.to_csv("main_df.csv")


In [28]:
with open("hawaii40_osu/gis/oneline_lines.geojson") as f:
    geo = json.load(f)

# --- Extract coordinates from each feature
rows = []
for feat in geo["features"]:
    name = feat["properties"]["Name"]
    coords = feat["geometry"]["coordinates"]
    if not coords or len(coords) < 2:
        continue  # skip malformed
    lon0, lat0 = coords[0]
    lon1, lat1 = coords[-1]
    rows.append({
        "name": name,
        "lat0": lat0, "lon0": lon0,
        "lat1": lat1, "lon1": lon1
    })


geo_df = pd.DataFrame(rows)

# --- Merge with main_df
# Adjust key if your column name differs (e.g., branch_name)
merged = df.merge(geo_df, on="name", how="left")


# --- Save to new CSV
merged.to_csv("main_df_with_coords.csv", index=False)
print("✅ Saved main_df_with_coords.csv with lat/lon columns added.")

✅ Saved main_df_with_coords.csv with lat/lon columns added.


In [29]:
merged.head()

,name,bus0,bus1,ckt,branch_name,bus0_name,bus1_name,s_nom_MVA,conductor,MOT_C,...,RES_25C,RES_50C,CDRAD_in,RLo_ohm_per_ft,RHi_ohm_per_ft,Diameter_in,lat0,lon0,lat1,lon1
0,L0,1,5,1,ALOHA138 (1) TO HONOLULU138 (5) CKT 1,ALOHA138,HONOLULU138,228,795 ACSR 26/7 DRAKE,80,...,0.1166,0.1278,0.5540,0.000022,0.000024,1.108,21.335554,-157.919593,21.316548,-157.835955
1,L1,1,5,2,ALOHA138 (1) TO HONOLULU138 (5) CKT 2,ALOHA138,HONOLULU138,228,795 ACSR 26/7 DRAKE,80,...,0.1166,0.1278,0.5540,0.000022,0.000024,1.108,21.335554,-157.922392,21.316548,-157.838754
2,L2,1,22,1,ALOHA138 (1) TO WAIPAHU138 (22) CKT 1,ALOHA138,WAIPAHU138,172,556.5 ACSR 26/7 DOVE,75,...,0.1655,0.1816,0.4635,0.000031,0.000034,0.927,21.344246,-157.919593,21.408782,-157.969141
3,L3,1,25,1,ALOHA138 (1) TO EWA BEACH138 (25) CKT 1,ALOHA138,EWA BEACH138,215,795 ACSR 26/7 DRAKE,85,...,0.1166,0.1278,0.5540,0.000022,0.000024,1.108,21.344246,-157.930791,21.336064,-158.007585
4,L4,1,25,2,ALOHA138 (1) TO EWA BEACH138 (25) CKT 2,ALOHA138,EWA BEACH138,215,795 ACSR 26/7 DRAKE,85,...,0.1166,0.1278,0.5540,0.000022,0.000024,1.108,21.344246,-157.933590,21.336064,-158.010384
